In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
X_test=np.load('X_test.npy')
y_test=np.load('y_test.npy')

In [3]:
channels=22
class CNNUnit(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(CNNUnit, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

class CNNLayer(nn.Module):
    def __init__(self, in_channels, out_channels, num_units, stride=1):
        super(CNNLayer, self).__init__()
        units = []
        for _ in range(num_units):
            units.append(CNNUnit(in_channels, out_channels, stride))
            in_channels = out_channels  # 更新输入通道数
        self.units = nn.Sequential(*units)

    def forward(self, x):
        return self.units(x)

class TransformerBlock(nn.Module):
    def __init__(self, feature_size, num_heads, dropout_rate):
        super(TransformerBlock, self).__init__()
        self.transformer = nn.TransformerEncoderLayer(
            d_model=feature_size,
            nhead=num_heads,
            dropout=dropout_rate
        )

    def forward(self, x):
        return self.transformer(x)

class HybridCNNTransformer(nn.Module):
    def __init__(self):
        super(HybridCNNTransformer, self).__init__()
        # CNN Unit (assuming input channels is 1 for STFT)
        self.cnn_unit = CNNLayer(in_channels=channels, out_channels=16, num_units=3)
        
        # CNN and Transformer layers
        self.cnn1 = CNNLayer(in_channels=16, out_channels=24, num_units=1, stride=2)
        self.transformer1 = TransformerBlock(feature_size=24, num_heads=8, dropout_rate=0.1)
        
        self.cnn2 = CNNLayer(in_channels=24, out_channels=32, num_units=1, stride=2)
        self.transformer2 = TransformerBlock(feature_size=32, num_heads=8, dropout_rate=0.1)
        
        # Pooling layer
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Fully connected layers
        self.fc1 = nn.Linear(32, 512)
        self.fc2 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()

    def forward(self, x):
        # Initial CNN processing
        x = self.cnn_unit(x)
        
        # First CNN and Transformer stage
        x = self.cnn1(x)
        x = x.flatten(2)  # Flatten the CNN features for the transformer
        x = x.permute(2, 0, 1)  # Reshape for transformer (Seq, Batch, Feature)
        x = self.transformer1(x)
        x = x.permute(1, 2, 0).unsqueeze(-1)  # Reshape back (Batch, Feature, Seq, 1)
        
        # Second CNN and Transformer stage
        x = self.cnn2(x)
        x = x.flatten(2)  # Flatten the CNN features for the transformer
        x = x.permute(2, 0, 1)  # Reshape for transformer
        x = self.transformer2(x)
        x = x.permute(1, 2, 0).unsqueeze(-1)  # Reshape back
        
        # Pooling and fully connected layers
        x = self.avg_pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Create the model
model = HybridCNNTransformer()
print(model)

HybridCNNTransformer(
  (cnn_unit): CNNLayer(
    (units): Sequential(
      (0): CNNUnit(
        (conv): Conv2d(22, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (1): CNNUnit(
        (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (2): CNNUnit(
        (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
    )
  )
  (cnn1): CNNLayer(
    (units): Sequential(
      (0): CNNUnit(
        (conv): Conv2d(16, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, 

In [4]:
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [6]:
test_data = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())
batch_size = 64
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

criterion = nn.CrossEntropyLoss()

In [7]:
def test(model, criterion, test_loader):
    model.eval()  # 将模型设置为评估模式
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # 关闭梯度计算
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            
            # 获取每个样本的预测结果（最大概率的类别）
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    test_accuracy = correct / total
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

# 加载最佳模型参数
model.load_state_dict(torch.load('best_model.pth'))



<All keys matched successfully>

In [8]:

# 在测试集上评估模型
test(model, criterion, test_loader)

Test Loss: 1.0907, Test Accuracy: 0.5718
